__Завгородняя Марина Игоревна Группа MADE-DS-22__ 

Второе домашнее задание по курсу "Продвинутое машинное обучение"

In [83]:
import pickle
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from scipy.stats import spearmanr, kendalltau
from scipy.special import logit, expit

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression

## Read & filter data

1) Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
* взять в тренировочный набор турниры с dateStart из 2019 года; 
* в тестовый — турниры с dateStart из 2020 года.

In [2]:
with open("chgk/tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)
with open("chgk/players.pkl", "rb") as f:
    players = pickle.load(f)
with open("chgk/results.pkl", "rb") as f:
    results = pickle.load(f)

In [3]:
with open("chgk/results.pkl", "rb") as f:
    results = pickle.load(f)

In [4]:
for i, t in tournaments.items():
    print(i, t)
    break

1 {'id': 1, 'name': 'Чемпионат Южного Кавказа', 'dateStart': '2003-07-25T00:00:00+04:00', 'dateEnd': '2003-07-27T00:00:00+04:00', 'type': {'id': 2, 'name': 'Обычный'}, 'season': '/seasons/1', 'orgcommittee': [], 'synchData': None, 'questionQty': None}


In [5]:
df_tournaments = pd.DataFrame(tournaments.values())

In [6]:
def filter_tournaments_and_results(df_tournaments):
    """Оставляем только турниры:
    1) 2019-2020 годов
    2) в которых есть информация о составе команд
    3) в результатах которых есть ответы в поле mask
    Возвращаем датафреймы с турнирами для трейна и теста: train_tournaments, test_tournaments 
    """
    train_tournaments = df_tournaments[df_tournaments.dateStart.str.startswith("2019")]
    test_tournaments = df_tournaments[df_tournaments.dateStart.str.startswith("2020")]
    
    tour_ids_to_del = []
    for key in [key for key in results]:
        teams = results[key]
        if teams is None or len(teams)==0: 
            del results[key]
            tour_ids_to_del.append(key)
            continue

        for team in teams:
            if 'mask' not in team or team['mask'] is None or len(team['mask'])==0 or team['mask']=='X' or team['mask']=='?':
                tour_ids_to_del.append(key)

    train_tournaments = train_tournaments[~train_tournaments.id.isin(tour_ids_to_del)].copy()
    test_tournaments = test_tournaments[~test_tournaments.id.isin(tour_ids_to_del)].copy()    
    
    print(f"Train tournaments shape: {train_tournaments.shape}, Test tournaments shape: {test_tournaments.shape}")
    return train_tournaments, test_tournaments

In [7]:
df_train_tournaments, df_test_tournaments = filter_tournaments_and_results(df_tournaments)

Train tournaments shape: (671, 9), Test tournaments shape: (169, 9)


## Baseline model

2) Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

### Dataset for Baseline model

In [136]:
def create_dataset(df):
    """
    Собираем датасет для обучения линейной модели:
    Результат ответа `result` на каждый вопрос `qid` присваиваем каждому игроку `player_id` в команде.
    :param df: датафрейм с турнирами на вход
    :returns: dataframe с колонками:
    | team_id, player_id, qid, result {0, 1} |
    """
    df["questions_total"] = df.questionQty.apply(lambda x: sum(x.values()))
    new_rows = []
    qid = 0
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        tourn_id = row["id"]
        tourn_result = results[tourn_id]
        questions_total = row["questions_total"]
        for team in tourn_result:
            team_id = team["team"]["id"]
            team_name = team["team"]["name"]
            team_questions_total = int(team["questionsTotal"])
            team_mask = team["mask"].replace("X", "0").replace("?", "0").replace("-", "0")
            team_mask_len = len(team_mask)
            if team_mask_len != questions_total:
                continue # Не включаем записи, где количество ответов в mask не равно количеству вопросов в турнире в questionQty
            for player in team["teamMembers"]:
                player_id = player["player"]["id"]
                player_name = player["player"]["surname"] +' '+ player["player"]["name"] +' '+ player["player"]["patronymic"]
                for i, q in enumerate(team_mask):
                    new_rows.append([tourn_id, team_id, player_id, qid + i, int(q)])
        qid += questions_total
    return pd.DataFrame(new_rows, columns=["tourn_id", "team_id", "player_id", "qid", "result"])

In [137]:
test_data = create_dataset(df_test_tournaments)
train_data = create_dataset(df_train_tournaments)

print(f"""Размер обучающей выборки: {train_data.shape}, 
      уникальных игроков: {train_data.player_id.nunique()}, 
      уникальных вопросов: {train_data.qid.nunique()}""")

# train_data.to_pickle("chgk/train_data.p")
# test_data.to_pickle("chgk/test_data.p")

100%|██████████| 671/671 [00:26<00:00, 25.34it/s]


Размер обучающей выборки: (17798678, 5), 
      уникальных игроков: 57357, 
      уникальных вопросов: 32987


In [10]:
train_data.head(2)

,team_id,player_id,qid,result
0,45556,6212,0,1
1,45556,6212,1,1


### Baseline logistic regression model

- Идея для Baseline модели - закодировать уникальные идентификаторы игроков для признаков, а в качестве таргета использовать результат ответа на вопрос. Так мы бы обучили модель бинарной классификации совсем не учитывая сам вопрос. 
- Чтобы хоть как-то учитывать вопросы, добавим в признаки закодированный идентификатор вопроса.
- После обучения линейной модели бинарной классификации на таких признаках мы получим веса для каждого игрока, которые означают вклад в вероятность успеха ответа на вопрос. На этом можно построить простую рейтинговую систему.

In [160]:
OHE = OneHotEncoder(handle_unknown='ignore')
X_train = OHE.fit_transform(train_data[['player_id', 'qid']])
X_test = OHE.transform(test_data[['player_id', 'qid']])
y_train = train_data['result']
y_test = test_data['result']

# Количество колонок после OHE должно соответствовать сумме кол-во уникальных игроков + кол-во уникальных вопросов
assert X_train.shape[1]==train_data.player_id.nunique() + train_data.qid.nunique()

model = LogisticRegression(penalty='l2', solver='liblinear').fit(X_train, y_train)
player_weights = model.coef_[0, :train_data.player_id.nunique()]
player_weight_map = dict(zip(OHE.categories_[0], player_weights))
player_weights.min(), player_weights.max()

(-4.156928678259833, 4.157537008862589)

### Top-10 players Baseline

In [112]:
best_players_baseline = list({k: v for k, v in sorted(player_weight_map.items(), key=lambda item: -item[1])}.keys())[:10]
for id in best_players_baseline:
    pl = players[id]
    fullname = pl["surname"]+" "+pl['name']+" "+pl["patronymic"]
    weight = round(player_weight_map[id], 3)
    print(f"{fullname} - {weight}")
    

Руссо Максим Михайлович - 4.158
Брутер Александра Владимировна - 4.031
Семушин Иван Николаевич - 3.986
Савченков Михаил Владимирович - 3.898
Спешков Сергей Леонидович - 3.818
Сорожкин Артём Сергеевич - 3.817
Мереминский Станислав Григорьевич - 3.7
Левандовский Михаил Ильич - 3.641
Прокофьева Ирина Сергеевна - 3.6
Николенко Сергей Игоревич - 3.582


## Calculate correlations for Baseline model

3) Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [12]:
true_test_results = {}
for _, row in tqdm(df_test_tournaments.iterrows(), total=df_test_tournaments.shape[0]):
    tourn_id = row["id"]
    tourn_result = results[tourn_id]
    questions_total = row["questions_total"]
    true_test_results[tourn_id]=[]
    for team in tourn_result:
        team_id = team["team"]["id"]
        team_questions_total = int(team["questionsTotal"])
        true_test_results[tourn_id].append([team_id, team_questions_total])
    true_test_results[tourn_id] = np.array(true_test_results[tourn_id])

100%|██████████| 169/169 [00:00<00:00, 2643.02it/s]


* Предположение: Команда ранжируется усредненным весов игроков команды. Чем больше вес, тем выше она в турнире.

In [94]:
def create_test_tourn_results(df, results, model_):
    if len(model_.coef_.shape)==1:
        player_weights = model_.coef_[:train_data.player_id.nunique()]
    else:
        player_weights = model_.coef_[0, :train_data.player_id.nunique()]
    player_weight_map = dict(zip(OHE.categories_[0], player_weights))
    
    predict_test_results = {}
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        tourn_id = row["id"]
        tourn_result = results[tourn_id]
        questions_total = row["questions_total"]
        predict_test_results[tourn_id]=[]
        for team in tourn_result:
            team_id = int(team["team"]["id"])
            team_weight = 0
            for player in team["teamMembers"]:
                player_id = player["player"]["id"]
                team_weight += player_weight_map.get(player_id, 0)
            if len(team["teamMembers"]) != 0:
                team_weight = team_weight / len(team["teamMembers"])
            predict_test_results[tourn_id].append([team_id, team_weight])
        predict_test_results[tourn_id] = np.array(predict_test_results[tourn_id])
    return predict_test_results

In [78]:
# Корреляции
def calc_correlations(df, results, model_, true_test_results):
    all_spearman_corr = []
    all_kendall_corr = []
    predict_test_results = create_test_tourn_results(df, results, model_)
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        tourn_id = row["id"]
        spearman_corr = spearmanr(true_test_results[tourn_id][:, 1], predict_test_results[tourn_id][:, 1])[0]
        kendall_corr = kendalltau(true_test_results[tourn_id][:, 1], predict_test_results[tourn_id][:, 1])[0]
        all_spearman_corr.append(spearman_corr)
        all_kendall_corr.append(kendall_corr)
    return np.nanmean(all_spearman_corr), np.nanmean(all_kendall_corr)

spearman_corr_mean, kendall_corr_mean = calc_correlations(df_test_tournaments, results, model, true_test_results)
print(f"Корреляция Спирмена: {spearman_corr_mean}, Корреляция Кендалла: {kendall_corr_mean}")

100%|██████████| 169/169 [00:00<00:00, 1022.90it/s]

Корреляция Спирмена: 0.7647727309624819, Корреляция Кендалла: 0.6116798808927042


## EM scheme

4) Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


ЕМ схема:
* Вводим скрытую переменную $z_i$ - ответ игрока, при условии, что он играет в данной команде.
* Начальное приближение для $z_i$ на нулевом шаге - ответы Baseline модели.
* На Е-шаге: \
    Обновляем предсказания ответа игрока: если команда не ответила на вопрос, то $z_i = 0$, \
    если ответила, то $z_i = P(x_i)/(1- prod_{x_i \in T}(1 - P(x_i)))$
* На М-шаге: \
    Максимизируем матожидание правдоподобия с помощью обучения линейных регрессий на ответах, полученных на Е-шаге.

In [8]:
model = LogisticRegression(penalty='l2', solver='liblinear').fit(X_train, y_train)
p_predict = model.predict_proba(X_train)[:, 1]

In [16]:
train_data.head(2)

,team_id,player_id,qid,result
0,45556,6212,0,1
1,45556,6212,1,1


In [98]:
N = 5
predict_proba = model.predict_proba(X_train)[:, 1]
for i in tqdm(range(N)):
    # E-шаг
    train_data["p"] = predict_proba
    train_data["1_p"] = 1 - predict_proba
    z_i_update = (train_data["p"]/(1 - train_data.groupby(["team_id","qid"])["1_p"].transform('prod')))*train_data["result"]
    z_i_update = np.clip(z_i_update, 1e-6, 1-1e-6)

    # М-шаг
    linreg_model = LinearRegression().fit(X_train, logit(z_i_update))
    predict_proba = expit(linreg_model.predict(X_train))

    # Корреляции
    spearmn, kendl = calc_correlations(df_test_tournaments, results, linreg_model, true_test_results)
    print(f"{i}: Корреляция Спирмена: {spearmn}, Корреляция Кендалла: {kendl}")
    

100%|██████████| 169/169 [00:00<00:00, 1242.32it/s]A

 20%|██        | 1/5 [03:30<14:01, 210.29s/it]

0: Корреляция Спирмена: 0.7634524827414099, Корреляция Кендалла: 0.6105569462692911



100%|██████████| 169/169 [00:00<00:00, 1029.09it/s]A

 40%|████      | 2/5 [06:59<10:29, 209.89s/it]

1: Корреляция Спирмена: 0.7632969041507203, Корреляция Кендалла: 0.609784774029013



100%|██████████| 169/169 [00:00<00:00, 1245.37it/s]A

 60%|██████    | 3/5 [10:34<07:03, 211.54s/it]

2: Корреляция Спирмена: 0.7641327555017509, Корреляция Кендалла: 0.6094185322439536



100%|██████████| 169/169 [00:00<00:00, 1230.56it/s]A

 80%|████████  | 4/5 [14:20<03:35, 215.93s/it]

3: Корреляция Спирмена: 0.7658149967789204, Корреляция Кендалла: 0.6103906410460618



100%|██████████| 169/169 [00:00<00:00, 1045.42it/s]A

100%|██████████| 5/5 [17:54<00:00, 214.81s/it]

4: Корреляция Спирмена: 0.7655817555873573, Корреляция Кендалла: 0.6103729560354443


### Выводы

- После применения ЕМ-алгоритма удалось добиться улучшения метрик корреляций
- В сравнении с Baseline-ом, который не учитывает результаты в команде, ЕМ-алгоритм дает лучшие результаты.

## Top tournaments

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)?

Сложность турнира рассчитываем как среднюю сложность вопросов турнира.

### Для Baseline модели

In [162]:
quest_weights = model.coef_[0, train_data.player_id.nunique():]
df_for_raiting = train_data.drop_duplicates(["tourn_id", "qid"])[["tourn_id"]]
df_for_raiting["weight"] = quest_weights
res_for_rating = df_for_raiting.groupby("tourn_id").mean()

print("Топ-20 самых сложных турниров: \n")
for id in res_for_rating.sort_values("weight").index[:20]:
    print(tournaments[id]["name"])

print("\n")

print("Топ-20 самых легких турниров: \n")
for id in res_for_rating.sort_values("weight").index[-20:]:
    print(tournaments[id]["name"])

Топ-20 самых сложных турниров: 

Чемпионат Санкт-Петербурга. Первая лига
Угрюмый Ёрш
Синхрон высшей лиги Москвы
Первенство правого полушария
Воображаемый музей
Записки охотника
Кубок городов
Знание – Сила VI
Ускользающая сова
Чемпионат России
Чемпионат Минска. Лига А. Тур четвёртый
Чемпионат Мира. Этап 2 Группа С
All Cats Are Beautiful
Чемпионат Мира. Этап 2. Группа В
VERSUS: Коробейников vs. Матвеев
Антибинго
Кубок Москвы
Львов зимой. Адвокат
Зеркало мемориала памяти Михаила Басса
Чемпионат Мира. Этап 1. Группа С


Топ-20 самых легких турниров: 

Лига вузов. IV тур
Школьный Синхрон-lite. Выпуск 3.1
Школьный Синхрон-lite. Выпуск 2.3
Второй тематический турнир имени Джоуи Триббиани
Школьная лига. II тур.
Межфакультетский кубок МГУ. Отбор №4
Малый кубок Физтеха
(а)Синхрон-lite. Лига старта. Эпизод X
Школьная лига. III тур.
Школьная лига
(а)Синхрон-lite. Лига старта. Эпизод VI
Школьная лига. I тур.
(а)Синхрон-lite. Лига старта. Эпизод IV
Студенческий чемпионат Калининградской области
Синх

### Для ЕМ модели

In [148]:
quest_weights = linreg_model.coef_[train_data.player_id.nunique():]
df_for_raiting = train_data.drop_duplicates(["tourn_id", "qid"])[["tourn_id"]]
df_for_raiting["weight"] = quest_weights
res_for_rating = df_for_raiting.groupby("tourn_id").mean()

In [157]:
print("Топ-20 самых сложных турниров: \n")
for id in res_for_rating.sort_values("weight").index[:20]:
    print(tournaments[id]["name"])

Топ-20 самых сложных турниров: 

Чемпионат Санкт-Петербурга. Первая лига
Угрюмый Ёрш
Синхрон высшей лиги Москвы
Воображаемый музей
Первенство правого полушария
Чемпионат Мира. Этап 2. Группа В
Чемпионат России
Знание – Сила VI
Чемпионат Мира. Этап 2. Группа А
Ускользающая сова
Записки охотника
Зеркало мемориала памяти Михаила Басса
Чемпионат Мира. Этап 3. Группа В
Чемпионат Минска. Лига А. Тур четвёртый
Чемпионат Мира. Этап 2 Группа С
Чемпионат Мира. Финал. Группа С
Львов зимой. Адвокат
Чемпионат Мира. Этап 1. Группа С
Чемпионат Мира. Этап 3. Группа С
Мемориал Дмитрия Коноваленко


In [158]:
print("Топ-20 самых легких турниров: \n")
for id in res_for_rating.sort_values("weight").index[-20:]:
    print(tournaments[id]["name"])

Топ-20 самых легких турниров: 

Школьный Синхрон-lite. Выпуск 2.3
(а)Синхрон-lite. Лига старта. Эпизод X
Малый кубок Физтеха
(а)Синхрон-lite. Лига старта. Эпизод IV
Школьный Синхрон-lite. Выпуск 3.3
Второй тематический турнир имени Джоуи Триббиани
Школьный Синхрон-lite. Выпуск 3.1
Межфакультетский кубок МГУ. Отбор №4
Школьная лига. II тур.
(а)Синхрон-lite. Лига старта. Эпизод III
(а)Синхрон-lite. Лига старта. Эпизод IX
(а)Синхрон-lite. Лига старта. Эпизод VII
Школьная лига
Школьный Синхрон-lite. Выпуск 2.5
Школьная лига. I тур.
Школьная лига. III тур.
Студенческий чемпионат Калининградской области
(а)Синхрон-lite. Лига старта. Эпизод V
Синхрон Лиги Разума
One ring - async


### Выводы

* Для ЕМ модели результаты сложности турниров совпадают с ожидаемыми: 
    * Легкие турниры - это турниры начального уровня, школьные
    * Сложные турниры в основном включают в себя Мировые этапы
* Для Baseline модели результаты для Топ-20 сложных турниров не совпадают с ожидаемыми - в них очень мало мировых турниров.
* Можно сделать вывод, что ЕМ-алгоритм помог нам выстроить не только рейтинг игроков, но рейтинг сложности турниров.